In [11]:
from jira import JIRA, Project
from jira.resources import Attachment
import os
from dotenv import load_dotenv
import boto3

load_dotenv()

True

In [ ]:
jira = JIRA(server='https://jira.valor6800.com', basic_auth=(os.getenv('JIRA_USERNAME'), os.getenv('JIRA_PASSWORD')))
issues = jira.search_issues('project = Hardware AND assignee = Empty AND status = "Ready to Fabricate"  and Machinery = "CNC Router"')
attachment: Attachment = issues[0].get_field('attachment')[0]
s3 = boto3.resource('s3')
bucket = s3.Bucket('autocam-attachments')
if not any(obj.key == attachment.filename for obj in bucket.objects.all()):
    bucket.put_object(Key=attachment.filename, Body=attachment.get())

s3.Object(bucket_name='autocam-attachments', key='Test25.step')

In [42]:
Epic = jira.issue(issues[0].get_field('customfield_10110')).get_field('summary')
Name = issues[0].get_field('summary')
Quantity = int(issues[0].get_field('customfield_10206'))
Ticket = issues[0].key
Material = str(issues[0].get_field('customfield_10202'))
Thickness = float(str(issues[0].get_field('customfield_10207')))

In [43]:
Epic, Name, Quantity, Ticket, Material, Thickness

('VR26A-1000A Chassis',
 'ManshikRonithCopyTest',
 10,
 'HAR-1035',
 'AL 6061',
 0.25)

# Postgres

In [29]:
import psycopg2

In [33]:
conn = psycopg2.connect(
            dbname="postgres",
            user="zachsharma",
            host="localhost",
            port="5432"
        )


In [34]:
cur = conn.cursor()

cur.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
""")

tables = cur.fetchall()
for t in tables:
    print(t[0])


account
box_tubes
part_categories
parts
parts_to_plates
plates
session
user
verification


In [31]:
cur = conn.cursor()

cur.execute("SELECT * FROM parts;")
rows = cur.fetchall()

for row in rows:
    print(row)

cur.close()
conn.close()

# Sql Alchemy

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, Table, select, MetaData

engine = create_engine("postgresql+psycopg2://zachsharma@localhost/postgres")

metadata = MetaData()
part_categories = Table("part_categories", metadata, autoload_with=engine)
plates = Table('parts', sqlalchemy.MetaData(), autoload_with=engine)

stmt = select(part_categories).where(
    part_categories.c.material == Material,
    part_categories.c.thickness == Thickness
)

with engine.begin() as conn:  
    result = conn.execute(stmt).fetchone()

    if result is None:
        ins = (
            part_categories.insert()
            .values(material=Material, thickness=Thickness)
            .returning(part_categories.c.id)  
        )
        category_id = conn.execute(ins).scalar_one()
    else:
        category_id = result._mapping["id"]  

print(category_id)

1


In [ ]:
stmt = select(plates).where(plates.c.name == Name)
with engine.begin() as conn:
    result = conn.execute(stmt).fetchone()
    if result is None:
        conn.execute(plates.insert().values(
            name=Name,
            epic=Epic,
            ticket=Ticket,
            quantity=Quantity,
            category_id=category_id
        ))

# AutoCAM

In [6]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()
AUTOCAM_APIKEY = os.getenv("AUTOCAM_APIKEY")
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {AUTOCAM_APIKEY}"})
response = session.get("http://localhost:3000/api/pc")
response.json()

[]

In [8]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()
AUTOCAM_APIKEY = os.getenv("AUTOCAM_APIKEY")
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {AUTOCAM_APIKEY}"})
response = session.post("http://localhost:3000/api/pc", json={
    "material": "Alluminum",
    "thickness": .25
})
response.json()

{'id': 1}